In [ ]:
#| export
#|default_exp fab

In [ ]:
#| export
#|export
doc = """**fab - Open Source 'fabric' prompts made quickly available in SolveIt**

This module leverages over 200 open source LLM prompts that are available in
Daniel Miesller's 'fabric' project.

If you import as fab, Submit the following to see an overview of all the prompts:
`fab.p`

**HOW TO USE IT**

Most Common Syntax: `prompt="Your Prompt"` in one cell then `fab.p.pattern_name()` in another, where pattern_name is any of the 200+ available fabric patterns.

**MOST IMPORTANT AND USED OPTIONS AND FEATURES**

- **Variable Targeting**: Use `fab.p.pattern_name('variable_name')` to process content from a specific variable instead of the default 'prompt' variable.

- **Pattern Discovery**: Use `fab.p.help()` (an alias for suggest_pattern()) to get suggestions of which pattern to pick for your prompt.

- **Compression Feature**: Use `fab.compress()` after running a pattern to save tokens by marking the previous cell as skipped and compressing the output to a new note.

- **Default Variable**: Most patterns work with a variable called 'prompt' by default, making it easy to process your main content.

**COMMON PATTERNS**

- For Summarizing Content: `fab.p.summarize()`
- For Explaining Code: `fab.p.explain_code()`
- For Analyzing Claims: `fab.p.analyze_claims()`
- For Extracting Wisdom from Text: `fab.p.extract_wisdom()`
- For Creating Quizzes: `fab.p.create_quiz()`
"""

__all__ = ['p', 'compress', 'doc', 'PatternFunction', 'FabricPatterns']
import os

class PatternFunction:
    def __init__(self, func, name, description):
        self.func = func
        self.name = name
        self.description = description
    
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
    def __repr__(self):
        return f"Fabric pattern: {self.name} - {self.description}"

class FabricPatterns:
    "Daniel Miesller's 'fabric' prompts made available in SolveIt"
    def __init__(self, patterns_path="/app/data/fabric/data/patterns"):
        from pathlib import Path    
        self.patterns_path = Path(patterns_path)
        self.pattern_descriptions = self._load_pattern_descriptions()
        if os.path.exists(patterns_path):
            self._load_patterns()
        else:
            self.help = print("Please get a copy of fabric in your /app/data folder - e.g. !cd /app/data; git clone --depth 1 https://github.com/danielmiessler/fabric.git")

    def _load_pattern_descriptions(self):
        suggest_pattern_file = self.patterns_path / "suggest_pattern" / "user.md"
        if suggest_pattern_file.exists():
            content = suggest_pattern_file.read_text()
            return self._parse_pattern_descriptions(content)
        return {}
    
    def _load_patterns(self):
        """Load all patterns from the patterns directory"""
        for pattern_dir in self.patterns_path.iterdir():
            if pattern_dir.is_dir():
                pattern_name = pattern_dir.name
                self._create_pattern_function(pattern_name, pattern_dir)
        if getattr(self, 'help', None) is None:
            self.help = self.suggest_pattern
    
    def _create_pattern_function(self, pattern_name, pattern_dir, run=True):
        from dialoghelper.core import read_msg, update_msg, add_msg, run_msg
        """Create a callable function for a pattern"""
        def pattern_function(prompt:str="prompt"):
            system_file = pattern_dir / "system.md"
            user_file = pattern_dir / "user.md"
            
            if not system_file.exists():
                return f"Pattern {pattern_name} not found (no system.md)"
            
            # Read system.md content
            prompt_content = system_file.read_text()
            
            # Add user.md if it exists and is not empty
            if user_file.exists():
                user_content = user_file.read_text().strip()
                if user_content:
                    prompt_content += "\n\n" + user_content
            
            # Add reference to text variable
            prompt_content += f"\n\n${prompt}"
            
            m = read_msg(0)['msg']
            update_msg(content=f"(From fab.p.{str(pattern_dir).split('/')[-1]} folded below)\n{prompt_content}", msg_type="prompt", i_collapsed=1, msgid=m['id'])
            if run: run_msg(msgid=m['id'])
            add_msg(content="fab.compress()", msg_type="code")
        
        # Wrap the function with PatternFunction
        description = self.pattern_descriptions.get(pattern_name, f"Pattern: {pattern_name}")
        wrapped_function = PatternFunction(pattern_function, pattern_name, description)
        setattr(self, pattern_name, wrapped_function)
        
    def _parse_pattern_descriptions(self, content):
        descriptions = {}
        lines = content.split('\n')

        for i, line in enumerate(lines):
            if line.startswith('### ') and i + 1 < len(lines):
                pattern_name = line[4:]  # Remove '### '
                description = lines[i + 2].strip()
                descriptions[pattern_name] = description

        return descriptions      

    def _children(self):
        return [c for c in dir(self) if not (c.startswith('_') or c in ('get','docs'))]

    def __iter__(self): 
        yield from (getattr(self, name) for name in self._children() 
                    if isinstance(getattr(self, name),PatternFunction))        

    def __repr__(self):
        s = ''
        if self.__doc__: s += self.__doc__ + '\n'
        if self.pattern_descriptions: s += "\n".join([f"{k}: {v}" for k, v in self.pattern_descriptions.items()])
        return s

# Create the fabric patterns instance
p = FabricPatterns()

def compress():
    """Compress last message's prompt+output into a note (reduce context)"""
    from dialoghelper import read_msg, update_msg
    m = read_msg(-1)['msg']
    if m['msg_type'] == "prompt":
        update_msg(i_collapsed=1, o_collapsed=1, skipped=True, msgid=m['id'])
        lines = m['content'].split('\n')
        update_msg(content=f"# Prompt {lines[0].replace(' folded below', '')} for {lines[-1]}\n{m['output']}", msg_type="note", msgid=read_msg(0)['msg']['id'])